In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
import numpy as np
import random
import math

In [12]:
def stratifySample(features, labels, bs):
    batches = []
    pos = np.where(labels == 1)[0]
    neg = np.where(labels == 0)[0]
    random.shuffle(pos)
    random.shuffle(neg)

    pos_num = len(pos)
    neg_num = len(neg)

    print("positive:{}\tnegative:{}".format(pos_num, neg_num))

    p_r = pos_num / (neg_num + pos_num)
    n_r = 1-p_r


    bs_p = math.ceil(bs * p_r)
    bs_n = bs - bs_p

    batches_p = math.floor(pos_num/bs_p)
    batches_n = math.floor(neg_num/bs_n)


    for i in range(min(batches_p, batches_n)):
        bt_p_features = features[pos[i*bs_p : (i+1)*bs_p]]
        bt_p_labels = labels[pos[i*bs_p : (i+1)*bs_p]]

        bt_n_features = features[neg[i*bs_n : (i+1)*bs_n]]
        bt_n_labels = labels[neg[i*bs_n : (i+1)*bs_n]]

        bt_comb_features = np.concatenate((bt_p_features, bt_n_features), axis = 0)
        bt_comb_labels = np.concatenate((bt_p_labels, bt_n_labels), axis = 0)

        idx = list(range(bs))
        random.shuffle(idx)
        bt_comb_features, bt_comb_labels = bt_comb_features[idx], bt_comb_labels[idx]
        bt_comb_features, bt_comb_labels = map(torch.tensor, (bt_comb_features, bt_comb_labels))
        batches.append((bt_comb_features, bt_comb_labels))
    return batches

In [13]:
class CBR(nn.Module):
    def __init__(self, in_channel, out_channel, stride=1, kernel_size=3):
        super(CBR, self).__init__()
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_size=kernel_size, stride=stride)
        self.bn = nn.BatchNorm2d(out_channel)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = nn.functional.relu(x)
        return x

In [14]:
class CBR_LargeT(nn.Module):
    def __init__(self, input_channels=3, kernel_size=7, stride=1):
        super(CBR_LargeT, self).__init__()
        self.in_channels = 32
        self.kernel_size = kernel_size
        self.stride = stride
        
        self.CBR1 = CBR(input_channels, self.in_channels, self.stride, self.kernel_size)
        self.pool1 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        
        self.CBR2 = CBR(self.in_channels, self.in_channels*2, self.stride, self.kernel_size)
        self.pool2 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        
        self.CBR3 = CBR(self.in_channels*2, self.in_channels*4, self.stride, self.kernel_size)
        self.pool3 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        
        self.CBR4 = CBR(self.in_channels*4, self.in_channels*8, self.stride, self.kernel_size)
        self.pool4 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        
        self.CBR5 = CBR(self.in_channels*8, self.in_channels*16, self.stride, self.kernel_size)
        
        
        # classifier
        self.fc = nn.Linear(self.in_channels*16, 2)
        
    def forward(self, x):
        x = self.CBR1(x)
        x = self.pool1(x)
        
        x = self.CBR2(x)
        x = self.pool2(x)
        
        x = self.CBR3(x)
        x = self.pool3(x)
        
        x = self.CBR4(x)
        x = self.pool4(x)
        
        x = self.CBR5(x)
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))
        
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [15]:
class CBR_LargeW(nn.Module):
    def __init__(self, input_channels=3, kernel_size=7, stride=1):
        super(CBR_LargeW, self).__init__()
        self.in_channels = 64
        self.kernel_size = kernel_size
        self.stride = stride
        
        self.CBR1 = CBR(input_channels, self.in_channels, self.stride, self.kernel_size)
        self.pool1 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        
        self.CBR2 = CBR(self.in_channels, self.in_channels*2, self.stride, self.kernel_size)
        self.pool2 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        
        self.CBR3 = CBR(self.in_channels*2, self.in_channels*4, self.stride, self.kernel_size)
        self.pool3 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        
        self.CBR4 = CBR(self.in_channels*4, self.in_channels*8, self.stride, self.kernel_size)
        self.pool4 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        
        
        # classifier
        self.fc = nn.Linear(self.in_channels*8, 2)
        
    def forward(self, x):
        x = self.CBR1(x)
        x = self.pool1(x)
        
        x = self.CBR2(x)
        x = self.pool2(x)
        
        x = self.CBR3(x)
        x = self.pool3(x)
        
        x = self.CBR4(x)
        x = self.pool4(x)
        
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))
        
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [16]:
class CBR_Small(nn.Module):
    def __init__(self, input_channels=3, kernel_size=7, stride=1):
        super(CBR_Small, self).__init__()
        self.in_channels = 32
        self.kernel_size = kernel_size
        self.stride = stride
        
        self.CBR1 = CBR(input_channels, self.in_channels, self.stride, self.kernel_size)
        self.pool1 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        
        self.CBR2 = CBR(self.in_channels, self.in_channels*2, self.stride, self.kernel_size)
        self.pool2 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        
        self.CBR3 = CBR(self.in_channels*2, self.in_channels*4, self.stride, self.kernel_size)
        self.pool3 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        
        self.CBR4 = CBR(self.in_channels*4, self.in_channels*8, self.stride, self.kernel_size)
        self.pool4 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        
        
        # classifier
        self.fc = nn.Linear(self.in_channels*8, 2)
        
    def forward(self, x):
        x = self.CBR1(x)
        x = self.pool1(x)
        
        x = self.CBR2(x)
        x = self.pool2(x)
        
        x = self.CBR3(x)
        x = self.pool3(x)
        
        x = self.CBR4(x)
        x = self.pool4(x)
        
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))
        
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [17]:
class CBR_Tiny(nn.Module):
    def __init__(self, input_channels=3, kernel_size=5, stride=1):
        super(CBR_Tiny, self).__init__()
        self.in_channels = 64
        self.kernel_size = kernel_size
        self.stride = stride
        
        self.CBR1 = CBR(input_channels, self.in_channels, self.stride, self.kernel_size)
        self.pool1 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        
        self.CBR2 = CBR(self.in_channels, self.in_channels*2, self.stride, self.kernel_size)
        self.pool2 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        
        self.CBR3 = CBR(self.in_channels*2, self.in_channels*4, self.stride, self.kernel_size)
        self.pool3 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        
        self.CBR4 = CBR(self.in_channels*4, self.in_channels*8, self.stride, self.kernel_size)
        self.pool4 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))
        
        
        # classifier
        self.fc = nn.Linear(self.in_channels*8, 2)
        
    def forward(self, x):
        x = self.CBR1(x)
        x = self.pool1(x)
        
        x = self.CBR2(x)
        x = self.pool2(x)
        
        x = self.CBR3(x)
        x = self.pool3(x)
        
        x = self.CBR4(x)
        x = self.pool4(x)
        
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))
        
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [18]:
DATA = np.load("../../../dataset/diabetic-retinopathy-npy/Messidor1/data.npy")
LABEL = np.load("../../../dataset/diabetic-retinopathy-npy/Messidor1/label.npy")
LABEL[LABEL < 3] = 0
LABEL[LABEL >=3 ] = 1


print(DATA.shape, LABEL.shape)
print(np.min(DATA), np.max(DATA))

from collections import Counter

Counter(LABEL)

(700, 224, 224, 3) (700,)
0.0 1.0


Counter({1: 174, 0: 526})

In [19]:
# split into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(DATA, LABEL, test_size=0.2, random_state=42)


data = {}
label = {}
data['train'] = X_train
label['train'] = y_train
data['val'] = X_test
label['val'] = y_test

print(X_train.shape, X_test.shape)

(560, 224, 224, 3) (140, 224, 224, 3)


In [20]:
# convert to PIL image
from PIL import Image
from matplotlib import cm


PIL_data = {x:[Image.fromarray(np.uint8(data[x][i] * 255)).convert('RGB') for i in range(data[x].shape[0])] for x in ['train', 'val']}
dataset_sizes = {x: len(PIL_data[x]) for x in ['train', 'val']}

print(dataset_sizes)

{'train': 560, 'val': 140}


In [21]:
data_transforms = {
    'train': transforms.Compose([
#         transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [23]:
data_tensor = {x: torch.stack([data_transforms[x](PIL_data[x][i]) for i in range(len(PIL_data[x]))]) for x in ['train', 'val']}
label_tensor = {x: torch.Tensor(label[x]) for x in ['train', 'val']}

In [24]:
ds = TensorDataset(data_tensor['train'], label_tensor['train'])

In [25]:
# ds = {x: TensorDataset(data_tensor[x], label_tensor[x]) for x in ['train', 'val']}
# dl = {x: DataLoader(ds[x], batch_size=32, shuffle=True, num_workers=8) for x in ['train', 'val']}

bs = 8
dl = {x: stratifySample(data_tensor[x], label_tensor[x], bs) for x in ['train', 'val']}

positive:133	negative:427
positive:41	negative:99


In [27]:
model_ft = CBR_Tiny()
print(model_ft)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

PATH = "D:/project/shallowNN/saved_models/CBR/CBR_Tiny.pt"
model_ft.load_state_dict(torch.load(PATH))
model_ft.eval()

CBR_Tiny(
  (CBR1): CBR(
    (conv): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (pool1): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (CBR2): CBR(
    (conv): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (pool2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (CBR3): CBR(
    (conv): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1))
    (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (pool3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (CBR4): CBR(
    (conv): Conv2d(256, 512, kernel_size=(5, 5), stride=(1, 1))
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (pool4): 

CBR_Tiny(
  (CBR1): CBR(
    (conv): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (pool1): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (CBR2): CBR(
    (conv): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (pool2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (CBR3): CBR(
    (conv): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1))
    (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (pool3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (CBR4): CBR(
    (conv): Conv2d(256, 512, kernel_size=(5, 5), stride=(1, 1))
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (pool4): 

In [28]:
# evaluate on validation set
correct = 0
with torch.no_grad():
    for i, (inputs, labels) in enumerate(dl['val']):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels.data)

print("accuracy: {}/{}".format(correct, X_test.shape[0]))

accuracy: 74/140
